### 調整以下設定(windows本機環境)
- config資料夾中的elasticsearch.yaml中xpack.security.enabled: false
- 同上，network.host: 0.0.0.0

### 安裝套件
- `pip install elasticsearch`

In [2]:
from elasticsearch import Elasticsearch as es


`cd Downloads\elasticsearch-8.17.3\bin`
 
`elasticsearch.bat`

### 先mapping 超長沒事別點開

In [249]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

index_name = "poi_data"

# Mapping
mapping = {
  "mappings": {
    "properties": {
      "location_id": { "type": "keyword" },
      "gmap_location": { 
        "type": "text",
        "fields": {
            "keyword": { 
                "type": "keyword",
                "ignore_above": 256
            }
        }
      },
      "class": { "type": "keyword" },
      "address": { 
        "type": "text",
        "fields": {
            "keyword": { 
                "type": "keyword",
                "ignore_above": 256
            }
        }
       },
      "summary_1": { "type": "text" },
      "summary_2": { "type": "text" },
      "opening_hours": {
        "type": "object",
        "properties": {
          "星期三": {
            "type": "object",
            "properties": {
              "periods": {
                "type": "nested",
                "properties": {
                  "open": { "type": "keyword" },
                  "close": { "type": "keyword" }
                }
              }
            }
          },
          "星期四": {
            "type": "object",
            "properties": {
              "periods": {
                "type": "nested",
                "properties": {
                  "open": { "type": "keyword" },
                  "close": { "type": "keyword" }
                }
              }
            }
          },
          "星期五": {
            "type": "object",
            "properties": {
              "periods": {
                "type": "nested",
                "properties": {
                  "open": { "type": "keyword" },
                  "close": { "type": "keyword" }
                }
              }
            }
          },
          "星期六": {
            "type": "object",
            "properties": {
              "periods": {
                "type": "nested",
                "properties": {
                  "open": { "type": "keyword" },
                  "close": { "type": "keyword" }
                }
              }
            }
          },
          "星期日": {
            "type": "object",
            "properties": {
              "periods": {
                "type": "nested",
                "properties": {
                  "open": { "type": "keyword" },
                  "close": { "type": "keyword" }
                }
              }
            }
          },
          "星期一": {
            "type": "object",
            "properties": {
              "periods": {
                "type": "nested",
                "properties": {
                  "open": { "type": "keyword" },
                  "close": { "type": "keyword" }
                }
              }
            }
          },
          "星期二": {
            "type": "object",
            "properties": {
              "periods": {
                "type": "nested",
                "properties": {
                  "open": { "type": "keyword" },
                  "close": { "type": "keyword" }
                }
              }
            }
          }
        }
      },
      "entrance_fee": { "type": "keyword" },
      "stay_time": { "type": "keyword" },
      "tags": {
        "type": "keyword"
      },
      "all_tags":{
        "type":"keyword"
      },
      "gmap_coordinates": {
        "type": "geo_point"
      }
    }
  }
}


if es.indices.exists(index=index_name):
    es.options(ignore_status=[400, 404]).indices.delete(index=index_name)


es.indices.create(index=index_name, body=mapping)

print("Elasticsearch Mapping 已更新")


Elasticsearch Mapping 已更新


### 把資料丟進去

In [250]:
import json

with open("../RAW_DATA/E_838_location_info.json", "r", encoding="utf-8") as f:
    pois = json.load(f)

for poi in pois:
    es.index(index="poi_data", document=poi)

print("POI 數據已導入 Elasticsearch！")


POI 數據已導入 Elasticsearch！


### 測試一下資料

In [167]:
response = es.search(index="poi_data", body={"query": {"match_all": {}}}, size=10)
for hit in response["hits"]["hits"]:
    print(hit["_source"])

{'location_id': '0x346885034084ae89:0x59a8b1eac5731c29', 'gmap_location': '白楊步道水簾洞', 'class': ['旅遊景點'], 'address': '972花蓮縣秀林鄉台8線166.5K號', 'summary_1': '可經由健行步道抵達的崖邊洞穴', 'summary_2': '熱門的步道，可穿越河谷和隧道前往有水自洞頂流下的狹小洞穴。', 'opening_hours': {'星期三': {'periods': [{'open': '00:00', 'close': '23:59'}]}, '星期四': {'periods': [{'open': '00:00', 'close': '23:59'}]}, '星期五': {'periods': [{'open': '00:00', 'close': '23:59'}]}, '星期六': {'periods': [{'open': '00:00', 'close': '23:59'}]}, '星期日': {'periods': [{'open': '00:00', 'close': '23:59'}]}, '星期一': {'periods': [{'open': '00:00', 'close': '23:59'}]}, '星期二': {'periods': [{'open': '00:00', 'close': '23:59'}]}}, 'entrance_fee': None, 'stay_time': None, 'tags': ['適合兒童'], 'gmap_coordinates': {'lat': 24.1805494, 'lon': 121.4743915}}
{'location_id': '0x345d4f79d991ebff:0x4b31be352c6025d8', 'gmap_location': '八斗子海濱公園', 'class': ['公園', '旅遊景點'], 'address': '202基隆市中正區北寧路21號', 'summary_1': '可飽覽大海和野花景緻的陡峭步道', 'summary_2': '鬱鬱蔥蔥的崎嶇山坡，可欣賞百合花盛開的美景，並設有階梯通往坐擁漁港和東海風光的瞭望台。', '

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7968\3464216612.py:1: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="poi_data", body={"query": {"match_all": {}}}, size=10)


#### 1. 基本地點檢索

In [123]:
from elasticsearch import Elasticsearch

# 連接 Elasticsearch
es = Elasticsearch("http://localhost:9200")

# 測試地點名稱檢索的 Query
test_queries = [
    "阿里山",  # 應匹配 "台北101購物中心"
    "故宮",  # 應匹配 "國立故宮博物院"
    "南故宮",  # 應匹配 "故宮南院"
    "士林夜市",  # 應匹配 "士林夜市"
    "水庫"  # 應匹配 "觀音山"
]

# Elasticsearch 查詢函式
def search_poi(query):
    es_query = {
        "size": 5,  # 避免 DeprecationWarning，將 size 移到 body 內
        "query": {
            "bool": {
                "should": [
                    {"match_phrase": {"gmap_location": query}},  # 完全匹配
                    {"wildcard": {"gmap_location": f"*{query}*"}},  # 部分匹配
                    {"match": {"gmap_location": {"query": query, "fuzziness": "AUTO"}}}  # 模糊匹配
                ],
                "minimum_should_match": 1
            }
        }
    }

    response = es.search(index="poi_data", body=es_query)
    results = []
    
    for hit in response["hits"]["hits"]:
        source = hit["_source"]
        result = {
            "gmap_location": source.get("gmap_location", "未知"),
            "address": source.get("address", "未知"),
            "location_id": source.get("location_id", "未知"),
            "經緯度": source.get("gmap_coordinates", "未知"),
            "匹配分數": hit["_score"],
            "匹配來源": []
        }
        
        # 確認匹配來源
        if query in source.get("gmap_location", ""):
            result["匹配來源"].append("gmap_location")

        if hit["_source"] is not None and "summary_1" in hit["_source"] and hit["_source"]["summary_1"] is not None:
            if query in hit["_source"]["summary_1"]:
                result["匹配來源"].append("summary_1")

        if hit["_source"] is not None and "summary_2" in hit["_source"] and hit["_source"]["summary_2"] is not None:
            if query in hit["_source"]["summary_2"]:
                result["匹配來源"].append("summary_2")

        results.append(result)

    return results

# 執行測試
for q in test_queries:
    print(f"\n🔍 `{q}` 搜索結果:")
    for res in search_poi(q):
        print(f"📌 {res['gmap_location']} | {res['location_id']} | 📍 {res['address']} | 🌍 經緯度: {res['經緯度']} | 🔢 匹配分數: {res['匹配分數']} | 🔎 來自: {', '.join(res['匹配來源'])}")



🔍 `阿里山` 搜索結果:
📌 阿里山火車站 | 0x346edf0462c35c37:0x24fc830686d49f4f | 📍 605嘉義縣阿里山鄉 | 🌍 經緯度: {'lat': 23.5100794, 'lon': 120.7794998} | 🔢 匹配分數: 22.357414 | 🔎 來自: gmap_location
📌 阿里山國家森林遊樂區 | 0x346edc20ac26153b:0xfff7f85ca03bbfd4 | 📍 605嘉義縣阿里山鄉 | 🌍 經緯度: {'lat': 23.5109539, 'lon': 120.8034992} | 🔢 匹配分數: 17.837023 | 🔎 來自: gmap_location
📌 阿里山森林鐵路車庫園區 | 0x346e9432844003c5:0xf0c1b488ea6cb9d2 | 📍 600嘉義市東區林森西路2號 | 🌍 經緯度: {'lat': 23.4863425, 'lon': 120.4409611} | 🔢 匹配分數: 16.9788 | 🔎 來自: gmap_location
📌 牛埔仔愛情大草原(阿里山18號愛情絲路) | 0x346eebb874fd20b3:0xc45658eb5b43f973 | 📍 602嘉義縣番路鄉觸口橋12 號 | 🌍 經緯度: {'lat': 23.4436554, 'lon': 120.5805834} | 🔢 匹配分數: 13.175251 | 🔎 來自: gmap_location
📌 阿里山國家風景區-管理處暨觸口遊客中心 | 0x346eebb003210cc9:0xfdab1aeab8baff2b | 📍 602嘉義縣番路鄉 | 🌍 經緯度: 未知 | 🔢 匹配分數: 12.701042 | 🔎 來自: gmap_location

🔍 `故宮` 搜索結果:
📌 國立故宮博物院 | 0x3442ac3acd404a7d:0x5d6d7018397a09c1 | 📍 111台北市士林區至善路二段221號 | 🌍 經緯度: {'lat': 25.1023554, 'lon': 121.5484925} | 🔢 匹配分數: 17.34105 | 🔎 來自: gmap_location
📌 國立故宮博物院南部院區 | 0x346e9bd42

#### 2. 用Gemini檢測關鍵字來產生Elasticsearch Query

In [4]:
import json
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import BadRequestError

# 連接 Elasticsearch (確認 Elasticsearch 實例已運行)
es = Elasticsearch("http://localhost:9200")

# 設定 Elasticsearch Index 名稱
INDEX_NAME = "poi_data"


def get_gmap_coordinates(location_name):
    """從 Elasticsearch 取得 `gmap_location` 的座標"""
    query = {
        "query": {"match": {"gmap_location": {"query": location_name, "fuzziness": "AUTO"}}},
        "size": 1,
    }
    response = es.search(index=INDEX_NAME, body=query)

    coordinates_list = []
    for hit in response["hits"]["hits"]:
        coords = hit["_source"].get("gmap_coordinates")
        if coords:
            coordinates_list.append(coords)

    if len(coordinates_list) == 1:
        return coordinates_list[0]  # 只有一個地點時，直接返回座標
    elif len(coordinates_list) > 1:
        return calculate_center_point(coordinates_list)  # 多個地點時，計算中心點
    return None


def calculate_center_point(coordinates_list):
    """計算多個地點的中心點"""
    latitudes = [coord["lat"] for coord in coordinates_list]
    longitudes = [coord["lon"] for coord in coordinates_list]
    center_lat = sum(latitudes) / len(latitudes)
    center_lon = sum(longitudes) / len(longitudes)
    return {"lat": center_lat, "lon": center_lon}


def time_period_to_time(time_period):
    """將時間段（例如 "上午", "下午", "晚上"）轉換為具體時間"""
    if time_period == "上午":
        return "10:00"
    elif time_period == "下午":
        return "14:00"
    elif time_period == "晚上":
        return "19:00"
    # Add more time period mapping as needed.
    return "00:00"  # default


def day_to_full_day(day):
    """
    將星期幾的簡稱或別稱轉換為全稱，並處理特殊情況。

    Args:
        day: 星期幾的簡稱、別稱或特殊字串。

    Returns:
        星期幾的全稱（例如："星期一"）或 None（如果無法識別）。
    """
    if "一" in day:
        return "星期一"
    elif "二" in day:
        return "星期二"
    elif "三" in day:
        return "星期三"
    elif "四" in day:
        return "星期四"
    elif "五" in day:
        return "星期五"
    elif "六" in day or "週末" in day or "周末" in day or "假日" in day:
        # 週末，周末 假日 包含星期六
        return "星期六"
    elif "日" in day or "天" in day:
        return "星期日"

    return None


def _build_location_query(parsed_query):
    """Build gmap_location or address query."""
    location_query = {"bool": {"should": []}}
    gmap_coordinates = None

    if "gmap_location" in parsed_query:
        location_query["bool"]["should"].append(
            {"match": {"gmap_location": {"query": parsed_query["gmap_location"], "fuzziness": "AUTO"}}}
        )
        gmap_coordinates = get_gmap_coordinates(parsed_query["gmap_location"])
        if not gmap_coordinates:
            print(f"警告 : gmap_location '{parsed_query['gmap_location']}' 未找到!")

    if "address" in parsed_query:
        if isinstance(parsed_query["address"], list):
            # 使用 terms 查詢
            location_query["bool"]["should"].extend(
                [{"match_phrase": {"address": {"query": addr}}} for addr in parsed_query["address"]]
            )
            # 找尋address是否帶有gmap_coordinates
            address_query = {"bool": {"should": [{"match_phrase": {"address": {"query": addr}}} for addr in parsed_query["address"]]}}
            response = es.search(
                index=INDEX_NAME, body={"query": address_query,"size":100}
            )
            address_coords = [
                hit["_source"].get("gmap_coordinates")
                for hit in response["hits"]["hits"]
                if hit["_source"].get("gmap_coordinates")
            ]

            if address_coords:
                gmap_coordinates = calculate_center_point(address_coords)
        else:
            # 使用 match_phrase 查詢(單一字串)
            location_query["bool"]["should"].append({"match_phrase": {"address": {"query": parsed_query["address"]}}})
            # 找尋address是否帶有gmap_coordinates
            address_query ={"match_phrase": {"address": {"query": parsed_query["address"]}}}
            response = es.search(
                index=INDEX_NAME,
                body={"query":address_query, "size": 100},
            )
            address_coords = [
                hit["_source"].get("gmap_coordinates")
                for hit in response["hits"]["hits"]
                if hit["_source"].get("gmap_coordinates")
            ]

            if address_coords:
                gmap_coordinates = calculate_center_point(address_coords)

    return location_query, gmap_coordinates


def _build_class_query(parsed_query):
    """Build class+gmap_location multi-field query ,
       so that documents that match both fields get higher score."""
    class_query = {
        "bool": {
            "should": [],
            "minimum_should_match": 1
        }
    }

    # 只要 user 有傳進 class
    if "class" in parsed_query:
        class_value = parsed_query["class"]

        # 不管是一個字串還是list，簡單處理成一個 list 來迭代
        if not isinstance(class_value, list):
            class_value = [class_value]

        for c in class_value:
            # 1) match 到 class 欄位 => boost=3
            class_query["bool"]["should"].append({
                "match": {
                    "class": {
                        "query": c,
                        "fuzziness": "AUTO",
                        "boost": 3.0
                    }
                }
            })
            # 2) match 到 gmap_location 欄位 => boost=1
            class_query["bool"]["should"].append({
                "match": {
                    "gmap_location": {
                        "query": c,
                        "fuzziness": "AUTO",
                        "boost": 1.0
                    }
                }
            })

    return class_query




def _build_class_gmap_location_query(parsed_query):
    """Build class gmap_location query."""
    class_gmap_location_query = {"bool": {"should": []}}
    if "class" in parsed_query:
        if isinstance(parsed_query["class"], list):
            class_gmap_location_queries = [
                {"match": {"gmap_location": {"query": c, "fuzziness": "AUTO", "boost": 1.5}}}
                for c in parsed_query["class"]
            ]  # gmap_location權重為1.5
            class_gmap_location_query["bool"]["should"].extend(class_gmap_location_queries)
        else:
            class_gmap_location_query["bool"]["should"].append(
                {"match": {"gmap_location": {"query": parsed_query["class"], "fuzziness": "AUTO", "boost": 1.5}}}
            )  # gmap_location權重為1.5
    return class_gmap_location_query

def _build_class_fallback_query(parsed_query):
    """Build class fallback query if class query doesn't have result."""
    if "class" not in parsed_query:
        return None
    class_fallback_query = {"bool": {"should": [],"minimum_should_match":1}} #增加minimum_should_match
    if isinstance(parsed_query["class"], list):
        class_fallback_queries = [
            {"match": {"summary_1": {"query": c, "fuzziness": "AUTO", "boost": 1}}}
            for c in parsed_query["class"]
        ] + [
            {"match": {"summary_2": {"query": c, "fuzziness": "AUTO", "boost": 1}}}
            for c in parsed_query["class"]
        ]  # summary權重為1.5
        class_fallback_query["bool"]["should"].extend(class_fallback_queries)
    else:
        class_fallback_query["bool"]["should"].extend(
            [
                {"match": {"summary_1": {"query": parsed_query["class"], "fuzziness": "AUTO", "boost": 1}}},
                {"match": {"summary_2": {"query": parsed_query["class"], "fuzziness": "AUTO", "boost": 1}}},
            ]
        )  # summary權重為1.5
    return class_fallback_query

def _build_tags_query(parsed_query):
    """Build tags query."""
    tags_query = {"bool": {"should": []}}
    if "tags" in parsed_query:
        for tag in parsed_query["tags"]:
            tags_query["bool"]["should"].append({"match": {"tags": {"query": tag, "fuzziness": "AUTO"}}})
    return tags_query

def _build_entrance_fee_query(parsed_query):
    """Build entrance_fee query."""
    if "entrance_fee" in parsed_query:
        return {"term": {"entrance_fee.keyword": parsed_query["entrance_fee"]}}
    return None

def _build_opening_hours_query(parsed_query):
    """Build opening_hours query."""
    if "opening_hours" in parsed_query:
        if isinstance(parsed_query["opening_hours"], list):
            day_of_week = day_to_full_day(parsed_query["opening_hours"][0])  # 取得星期幾
            time_of_day = time_period_to_time(parsed_query["opening_hours"][1])  # 取得時間
        elif isinstance(parsed_query["opening_hours"], dict):
            day_of_week = day_to_full_day(parsed_query["opening_hours"]["day"])  # 取得星期幾
            time_of_day = time_period_to_time(parsed_query["opening_hours"]["time"])  # 取得時間
        else:
            day_of_week = None
            time_of_day = None

        if day_of_week and time_of_day:
            return {
                "nested": {
                    "path": f"opening_hours.{day_of_week}.periods",
                    "query": {
                        "bool": {
                            "should": [
                                {
                                    "bool": {
                                        "must": [
                                            {
                                                "range": {
                                                    f"opening_hours.{day_of_week}.periods.open": {
                                                        "lte": time_of_day,
                                                        "format": "HH:mm",
                                                    }
                                                }
                                            },
                                            {
                                                "range": {
                                                    f"opening_hours.{day_of_week}.periods.close": {
                                                        "gte": time_of_day,
                                                        "format": "HH:mm",
                                                    }
                                                }
                                            },
                                        ]
                                    }
                                },
                                { #if no period or time out of periods.
                                    "bool":{
                                        "must_not":{
                                            "exists":{"field":f"opening_hours.{day_of_week}.periods"}
                                        }
                                    }
                                },
                                { #check 00:00 - 23:59 period.
                                    "bool":{
                                        "must": [
                                            {
                                                "term": {f"opening_hours.{day_of_week}.periods.open": "00:00"}
                                            },
                                            {
                                                "term": {f"opening_hours.{day_of_week}.periods.close": "23:59"}
                                            }
                                        ]
                                    }
                                }
                            ],
                            "minimum_should_match": 1,
                        }
                    },
                }
            }
    return None

def _build_geo_distance_query(parsed_query, gmap_coordinates):
    """Build geo_distance query."""
    if "geo_distance" in parsed_query and gmap_coordinates:
        return {
            "geo_distance": {
                "distance": parsed_query["geo_distance"],
                "gmap_coordinates": {"lat": gmap_coordinates["lat"], "lon": gmap_coordinates["lon"]},
            }
        }
    return None


def generate_es_query(parsed_query):
    """根據 Gemini 解析結果產生 Elasticsearch Query"""
    es_query = {"query": {"bool": {"must": [], "should": [], "filter": []}}}
    must_conditions = es_query["query"]["bool"]["must"]
    should_conditions = es_query["query"]["bool"]["should"]
    filter_conditions = es_query["query"]["bool"]["filter"]

    # 處理地點查詢 (gmap_location 或 address)
    location_query, gmap_coordinates = _build_location_query(parsed_query)
    if location_query["bool"]["should"]:
        # 將 address 的 match_phrase 直接放在 must
        for item in location_query["bool"]["should"]:
            must_conditions.append(item)

    # 處理 class 查詢
    class_query = _build_class_query(parsed_query)
    class_gmap_location_query = _build_class_gmap_location_query(parsed_query)
    class_fallback_query = _build_class_fallback_query(parsed_query)

    #加入在query body增加size
    query_class = {"query": class_query,"size":10}
    query_class_gmap = {"query": class_gmap_location_query,"size":10}

    if class_query["bool"]["should"]:
    # 如果確定要把 class_query 放進最終 must
        must_conditions.append(class_query)

    # # 檢查是否有class match到結果
    # response = es.search(index=INDEX_NAME, body=query_class)
    # if not response["hits"]["hits"]: # 如果class沒有找到任何資料
    #     # 處理 class 在 gmap_location 中搜尋
    #     if class_gmap_location_query["bool"]["should"]:
    #         should_conditions.append(class_gmap_location_query)
    #         # 處理 class 在 summary 中搜尋
    #         response = es.search(index=INDEX_NAME, body=query_class_gmap)
    #         if not response["hits"]["hits"] and class_fallback_query: #如果gmap_location也找不到，且有class_fallback_query才進行summary的搜尋
    #             should_conditions.append(class_fallback_query)
    #     elif class_fallback_query: # 如果class_gmap_location_query沒有，且有class_fallback_query，才進行summary搜尋
    #          should_conditions.append(class_fallback_query)
            

    # 處理 tags 查詢 (現在不是 nested query)
    tags_query = _build_tags_query(parsed_query)
    if tags_query["bool"]["should"]:
        should_conditions.append(tags_query)

    # 加入 entrance_fee 過濾條件
    entrance_fee_query = _build_entrance_fee_query(parsed_query)
    if entrance_fee_query:
        filter_conditions.append(entrance_fee_query)

    # 加入 opening_hours 過濾條件
    opening_hours_query = _build_opening_hours_query(parsed_query)
    if opening_hours_query:
        filter_conditions.append(opening_hours_query)

    # 加入 geo_distance 查詢
    geo_distance_query = _build_geo_distance_query(parsed_query, gmap_coordinates)
    if geo_distance_query:
        filter_conditions.append(geo_distance_query)

    # 若 must 和 filter 都沒有條件，則不輸出 bool 結構
    if not must_conditions and not filter_conditions and not should_conditions:
        es_query = {"query": {"match_all": {}}}
    elif not must_conditions and not should_conditions:
        es_query["query"] = {"bool": {"filter": filter_conditions}}
    elif not must_conditions and not filter_conditions:
        es_query["query"] = {"bool": {"should": should_conditions}}
    elif not filter_conditions and not should_conditions:
        es_query["query"] = {"bool": {"must": must_conditions}}
    elif not must_conditions:
        es_query["query"] = {"bool": {"should": should_conditions, "filter": filter_conditions}}
    elif not filter_conditions:
        es_query["query"] = {"bool": {"must": must_conditions, "should": should_conditions}}
    elif not should_conditions:
        es_query["query"] = {"bool": {"must": must_conditions, "filter": filter_conditions}}
    else:
        es_query["query"] = {"bool": {"must": must_conditions, "should": should_conditions, "filter": filter_conditions}}

    return es_query



def search_elasticsearch(parsed_query):
    """
    綜合式查詢函式：
      - Part A: Address / Location 處理
      - Part B: 其他過濾 (tags, entrance_fee, opening_hours, geo_distance)
      - Part C: class + gmap_location 一起查 (boost) → 若無 => fallback 查 summary
      - 組合最終 ES Query, 執行查詢, 根據score/min_output_count做過濾
    """

    # ------------------------ Part A: Address / Location ------------------------
    location_query, gmap_coordinates = _build_location_query(parsed_query)
    must_conditions = []
    should_conditions = []
    filter_conditions = []

    # 將 address 相關的搜尋條件加到 must
    if location_query["bool"]["should"]:
        for item in location_query["bool"]["should"]:
            must_conditions.append(item)

    # ------------------------ Part B: 其他 filters (tags, entrance_fee, opening_hours, geo_distance) ------------------------
    tags_query = _build_tags_query(parsed_query)
    if tags_query["bool"]["should"]:
        should_conditions.append(tags_query)

    entrance_fee_query = _build_entrance_fee_query(parsed_query)
    if entrance_fee_query:
        filter_conditions.append(entrance_fee_query)

    opening_hours_query = _build_opening_hours_query(parsed_query)
    if opening_hours_query:
        filter_conditions.append(opening_hours_query)

    geo_distance_query = _build_geo_distance_query(parsed_query, gmap_coordinates)
    if geo_distance_query:
        filter_conditions.append(geo_distance_query)

    # ------------------------ Part C: 「class + gmap_location」 + fallback 到 summary ------------------------
    class_value = parsed_query.get("class", None)
    if class_value:
        # Step C1: 建立 "class + gmap_location" 多欄位 should 查詢
        # 同時命中兩欄 → 得分相加更高
        class_gmap_query = {
            "bool": {
                "should": [
                    {
                        "match": {
                            "class": {
                                "query": class_value,
                                "fuzziness": "AUTO",
                                "boost": 3.0
                            }
                        }
                    },
                    {
                        "match": {
                            "gmap_location": {
                                "query": class_value,
                                "fuzziness": "AUTO",
                                "boost": 1.0
                            }
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        }

        # 先看 class+gmap_location 有沒有命中
        resp_c1 = es.search(
            index=INDEX_NAME,
            body={"query": class_gmap_query, "size": 10}
        )

        if resp_c1["hits"]["hits"]:
            # 如果有命中，就把它加到 must
            must_conditions.append(class_gmap_query)
        else:
            # Step C2: fallback 查 summary_1 / summary_2
            fallback_summary_query = {
                "bool": {
                    "should": [
                        {
                            "match": {
                                "summary_1": {
                                    "query": class_value,
                                    "fuzziness": "AUTO"
                                }
                            }
                        },
                        {
                            "match": {
                                "summary_2": {
                                    "query": class_value,
                                    "fuzziness": "AUTO"
                                }
                            }
                        }
                    ],
                    "minimum_should_match": 1
                }
            }
            resp_summary = es.search(
                index=INDEX_NAME,
                body={"query": fallback_summary_query, "size": 10}
            )
            if resp_summary["hits"]["hits"]:
                must_conditions.append(fallback_summary_query)
            else:
                print(f"『class={class_value}』在 class/gmap_location/summary 都無搜尋結果，略過 class 條件")
                # 不加任何 class 條件

    # ------------------------ Part D: 組合最終的 ES Query ------------------------
    es_query = {
        "query": {
            "bool": {
                "must": must_conditions,
                "should": should_conditions,
                "filter": filter_conditions
            }
        }
    }
    if not must_conditions and not should_conditions and not filter_conditions:
        es_query = {"query": {"match_all": {}}}

    print("🚀 產生的最終 ES Query:")
    print(json.dumps(es_query, indent=2, ensure_ascii=False))

    # ------------------------ Part E: 執行查詢 + score/min_output_count 過濾 ------------------------
    try:
        response = es.search(index=INDEX_NAME, body=es_query, size=100)
        hits = response["hits"]["hits"]

        filtered_results = []
        output_count = 0
        threshold_score = 0.9  # 相關性分數閾值(可依需求調整)
        min_output_count = 10  # 最少輸出數量(可依需求調整)

        if not hits:
            return filtered_results

        for hit in hits:
            score = hit["_score"]
            # 若已滿最少輸出數, 且分數又低於threshold, 就可以停止
            # (視情況保留或移除這段)
            if (output_count >= min_output_count and score < threshold_score) or (output_count >= len(hits)):
                break

            filtered_results.append(hit)
            output_count += 1

        return filtered_results

    except BadRequestError as e:
        print(f"❌ Elasticsearch BadRequestError: {e}")
        return []



if __name__ == "__main__":
    test_queries = [
        {"address": "台中市","class": "博物館"},        
    ]

    for query in test_queries:
        print(f"\n🔍 測試 Elasticsearch 查詢: {query}")
        results = search_elasticsearch(query)
        if results:
            for result in results:
                print(result["_source"])
        else:
            print("未找到任何符合條件的地點")



🔍 測試 Elasticsearch 查詢: {'address': '台中市', 'class': '博物館'}
🚀 產生的最終 ES Query:
{
  "query": {
    "bool": {
      "must": [
        {
          "match_phrase": {
            "address": {
              "query": "台中市"
            }
          }
        },
        {
          "bool": {
            "should": [
              {
                "match": {
                  "class": {
                    "query": "博物館",
                    "fuzziness": "AUTO",
                    "boost": 3.0
                  }
                }
              },
              {
                "match": {
                  "gmap_location": {
                    "query": "博物館",
                    "fuzziness": "AUTO",
                    "boost": 1.0
                  }
                }
              }
            ],
            "minimum_should_match": 1
          }
        }
      ],
      "should": [],
      "filter": []
    }
  }
}
{'location_id': '0x34693b47de0e09dd:0x798ad9adcb085701', 'gmap_location': '立法院議

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3704\2342564922.py:528: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=INDEX_NAME, body=es_query, size=100)
